In [ ]:
%pip install langchain pgvector psycopg2-binary tiktoken

In [13]:
import os
from langchain.llms import HuggingFacePipeline
from langchain.llms import HuggingFaceTextGenInference
from langchain.document_loaders import TextLoader
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores.pgvector import PGVector

In [14]:
# test our Llama-2 interface with langchain

llm = HuggingFaceTextGenInference(
    inference_server_url="http://34.70.31.13/",
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
)
llm("[INST] You are a garden who has been trained to provide helpful, respectful and honest answers about yourself and garden plants. Always answer as helpfully as possible, while being safe and keep your responses less than 200 words. Is salt good for asparagus?[/INST]")

" Salt can be beneficial for asparagus in small amounts. It helps to enhance the flavor of the asparagus and can also help to prevent the growth of harmful bacteria. However, it's important to note that too much salt can be harmful to asparagus and other plants, so it's best to use it sparingly. Additionally, it's important to ensure that the soil in which the asparagus is growing is well-drained and has a good balance of nutrients, as salt alone is not enough to support healthy growth."

In [15]:
# Load list of URLs -> kubernetes.io/docs/concepts/
file1 = open('./data/plant-urls.samples.txt', 'r')

loader = WebBaseLoader(file1.read().splitlines())
documents = loader.load()

In [16]:
# Chunk all the kubernetes concept documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

print("%s chunks in %s pages" % (len(docs), len(documents)))

77 chunks in 2 pages


In [ ]:
docs

In [17]:
# Load sentence transformer embeddings
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device":"cpu"} # use {"device":"cuda"} for distributed embeddings

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

In [18]:
# Connection string for connecting to Postgres
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver=os.environ.get("PGVECTOR_DRIVER", "psycopg2"),
    host=os.environ.get("PGVECTOR_HOST", "localhost"),
    port=int(os.environ.get("PGVECTOR_PORT", "5432")),
    database=os.environ.get("PGVECTOR_DATABASE", "postgres"),
    user=os.environ.get("PGVECTOR_USER", "postgres"),
    password=os.environ.get("PGVECTOR_PASSWORD", "secretpassword"),
)

In [19]:
COLLECTION_NAME = "plants-3"

db = PGVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

In [1]:
# A better way with distributed jobs on Kubernetes - Queue Docs
! sh ./queue-documents.sh -f './data/plant-urls.txt' -t plants-2

queuing documents for distributed processing...
---
File Path: ./data/plant-urls.txt
PubSub Topic: plants-2
---
publishing https://en.wikipedia.org/wiki/Abelia
messageIds:
- '9753628720817927'
publishing https://en.wikipedia.org/wiki/Abroma
messageIds:
- '9076825938252472'
publishing https://en.wikipedia.org/wiki/Abrus
messageIds:
- '9753249188562269'
publishing https://en.wikipedia.org/wiki/Abutilon
messageIds:
- '9753904180023155'
publishing https://en.wikipedia.org/wiki/Acaena
messageIds:
- '9077094802799671'
publishing https://en.wikipedia.org/wiki/Acalypha
messageIds:
- '9753527373857721'
publishing https://en.wikipedia.org/wiki/Acanthaceae
messageIds:
- '9077232016620252'
publishing https://en.wikipedia.org/wiki/Acanthodium
messageIds:
- '9753322369762277'
publishing https://en.wikipedia.org/wiki/Acantholimon
messageIds:
- '9753650600481543'
publishing https://en.wikipedia.org/wiki/Acanthopale
messageIds:
- '9077244049949260'
publishing https://en.wikipedia.org/wiki/Acanthophoeni

In [21]:
# A better way with distributed jobs on Kubernetes
! sh ./deploy-indexer.sh

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


job.batch/indexer-job unchanged
---
waiting for job to start...
error: timed out waiting for the condition


In [22]:
query = "Is salt good for asparagus?" # "Should I use gateway API in my app?"
docs = db.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")

Query: Is salt good for asparagus?
Retrieved documents: 4
Source:  https://en.wikipedia.org/wiki/Asparagus
Text:  Cultivation[edit]
See also: List of asparagus diseases
Since asparagus often originates in maritime habitats, it thrives in soils that are too saline for normal weeds to grow. Thus, a little salt was traditionally used to suppress weeds in beds intended for asparagus; this has the disadvantage that the soil cannot be used for anything else. Some places are better for growing asparagus than others. The fertility of the soil is a large factor. "Crowns" are planted in winter, and the first shoots appear in spring; the first pickings or "thinnings" are known as sprue asparagus. Sprue has thin stems.[25]
A breed of "early-season asparagus" that can be harvested two months earlier than usual was announced by a UK grower in early 2011.[26] This variety does not need to lie dormant and blooms at 7 °C (45 °F), rather than the usual 9 °C (48 °F). 

Source:  https://en.wikipedia.org/w

In [ ]:
docs

In [23]:
from langchain.prompts import PromptTemplate
prompt_template = """[INST] You are a helpful, respectful and honest assistant who is an expert in explaining Kubernetes concepts. Always answer as helpfully as possible, while being safe.
        Use the following pieces of context to answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer.
        
        {context}

        Question: {question}
        Answer:[/INST]"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [24]:
retriever = db.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=True
)

In [27]:
result = qa.run("When should I plant asparagus?")
print(result)



> Entering new RetrievalQA chain...

> Finished chain.
 Asparagus can be planted in the winter, typically in November or December, depending on your location. In colder climates, it's best to plant asparagus crowns, which are the roots of the plant, rather than seedlings. The first shoots will appear in the spring, and the first pickings, known as sprue asparagus, can be harvested in late spring or early summer. It's important to note that asparagus thrives in soils that are too saline for normal weeds to grow, and the fertility of the soil is a large factor. Additionally, some varieties of asparagus can be harvested two months earlier than usual, but this requires careful management and monitoring of the plants.
